In [1]:
# Library load: 
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import ast
import nltk
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate as evaluate
from surprise.model_selection import GridSearchCV
import pandas as pd
#It is necessary to install "surprise", sometimes needs to update Microsoft Visual C++ 
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV


nltk.download('stopwords')
stemmer = SnowballStemmer("english")
#from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sebas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pandas as pd
meta_path = 'train_meta_products.csv'
reviews_path = 'train_reviews_products.csv'

# Data frames load
meta = pd.read_csv(meta_path,encoding = "ISO-8859-1")
reviews = pd.read_csv(reviews_path,encoding = "ISO-8859-1")

In [3]:
# First of all we will drop the products duplicated on the 'asin' variable.
meta = meta.drop_duplicates(subset = ['asin'])

# Data Exploration

The first step in the developement of the recommender will be a data exploration of the data frames.

- Data frame information
- Number of missings per variable 
- Number of categories per variable

### Meta exploration

In [4]:
meta.head()

,Unnamed: 0,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,0,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),['Product Dimensions:\n \n8...,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,XCC Qi promise new spider snake preparing men'...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Magical Things I Really Do Do Too!,Christopher Manos,['Package Dimensions:\n \n8...,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,['For the professional or amateur magician. R...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Ashes to Ashes, Oranges to Oranges",Flickerlamp Publishing,['Package Dimensions:\n \n8...,"19,734,184inClothing,ShoesJewelry(",5 star,1940735033,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Aether & Empire #1 - 2016 First Printing Comic...,NaN,['Package Dimensions:\n \n1...,"10,558,646inClothing,Shoesamp;Jewelry(",5 star,1940967805,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN,$4.50,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186186 entries, 0 to 186689
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Unnamed: 0       186186 non-null  object
 1   title            186177 non-null  object
 2   brand            139666 non-null  object
 3   feature          123783 non-null  object
 4   rank             179848 non-null  object
 5   date             184503 non-null  object
 6   asin             186185 non-null  object
 7   imageURL         131803 non-null  object
 8   imageURLHighRes  131803 non-null  object
 9   description      15687 non-null   object
 10  price            17788 non-null   object
 11  also_view        11579 non-null   object
 12  also_buy         21636 non-null   object
 13  fit              4827 non-null    object
 14  details          883 non-null     object
 15  similar_item     317 non-null     object
 16  tech1            97 non-null      object
dtypes: object(

- The Meta data frame has the essencial information about the products. There are 16 variables and around 20k registers. Most of the variables are string type. 

In [6]:
print('---------Percentage of NA per variable------------')
print((meta.isna().sum()/len(meta)*100).sort_values(ascending=False))

---------Percentage of NA per variable------------
tech1              99.947902
similar_item       99.829740
details            99.525743
fit                97.407431
also_view          93.780950
description        91.574554
price              90.446113
also_buy           88.379363
feature            33.516484
imageURLHighRes    29.208963
imageURL           29.208963
brand              24.985767
rank                3.404123
date                0.903935
title               0.004834
asin                0.000537
Unnamed: 0          0.000000
dtype: float64


Analyzing the results we can make the following decisions:

1. The variables with a high percentaje of missing values wont be usefull for the model so we will drop them: these are tech1, similar_item, details, fit, description, price, also_view and also_buy.
2. The variables imageURL and imageURLHighRes would be very useful if our recommender used artificial vision techniques. However, it does not use them and therefore we will drop these variables.

In [7]:
print('\n--------- Number of categories per variable (unique values) ------------')
print(meta.nunique().sort_values())


--------- Number of categories per variable (unique values) ------------
tech1                  92
similar_item          316
details               832
fit                  2635
price                5120
date                 7001
also_view           10821
description         13077
brand               18513
also_buy            19333
feature             77008
imageURLHighRes     98757
imageURL            98830
title              175253
rank               178280
asin               186185
Unnamed: 0         186186
dtype: int64


Analyzing the results we can make the following decisions:
1. As most of the variables are objects (string) and all of them have a lot of unique values, we will use Natural Languaje Processing techniques to extract value from this data frame. 

### Meta precleaning
Following these decisions we can clean the Meta data frame.

In [8]:
def meta_precleaning(data):
    data = data.drop(['tech1', 'similar_item', 'details', 'fit', 'description', 'price', 'also_view' ,'also_buy'],axis = 1)
    data = data.drop(['imageURL','imageURLHighRes'],axis = 1)
    return data

meta = meta_precleaning(meta)
meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186186 entries, 0 to 186689
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  186186 non-null  object
 1   title       186177 non-null  object
 2   brand       139666 non-null  object
 3   feature     123783 non-null  object
 4   rank        179848 non-null  object
 5   date        184503 non-null  object
 6   asin        186185 non-null  object
dtypes: object(7)
memory usage: 11.4+ MB


We have cut memory usage in half

### Reviews exploration

In [9]:
reviews.head()

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,0,5.0,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,1,2.0,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,NaN,NaN
2,2,4.0,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN
3,3,2.0,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,NaN,NaN
4,4,3.0,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,NaN,NaN


In [10]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883340 entries, 0 to 883339
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      883340 non-null  int64  
 1   overall         883340 non-null  float64
 2   verified        883340 non-null  bool   
 3   reviewTime      883340 non-null  object 
 4   reviewerID      883340 non-null  object 
 5   asin            883340 non-null  object 
 6   reviewerName    883212 non-null  object 
 7   reviewText      882107 non-null  object 
 8   summary         882803 non-null  object 
 9   unixReviewTime  883340 non-null  int64  
 10  vote            79875 non-null   float64
 11  style           304511 non-null  object 
 12  image           28802 non-null   object 
dtypes: bool(1), float64(2), int64(2), object(8)
memory usage: 81.7+ MB


- The Reviews data frame has the information about the reviews of products. There are 12 variables and around 800k registers. 

In [11]:
print('---------Percentage of NA per variable------------')
print((reviews.isna().sum()/len(reviews)*100).sort_values(ascending=False))

---------Percentage of NA per variable------------
image             96.739421
vote              90.957615
style             65.527317
reviewText         0.139584
summary            0.060792
reviewerName       0.014490
Unnamed: 0         0.000000
overall            0.000000
verified           0.000000
reviewTime         0.000000
reviewerID         0.000000
asin               0.000000
unixReviewTime     0.000000
dtype: float64


Analyzing the results we can make the following decisions:
1. Just like we did with the Meta data frame, we can drop the variables with a high percentaje of missing values. 

In [12]:
print('\n--------- Number of categories per variable (unique values) ------------')
print(reviews.nunique().sort_values())


--------- Number of categories per variable (unique values) ------------
verified               2
overall                5
vote                 200
reviewTime          4608
unixReviewTime      4608
image              28296
style              34308
asin              186158
summary           388515
reviewerName      467704
reviewerID        749223
reviewText        768550
Unnamed: 0        883340
dtype: int64


Analyzing the results we can make the following decisions:
1. As overall is a float variable between 1 and 5 we can use it as an indicator of the products quality. 
2. We can use verified to check if the person who wrote the review really bought the item. 
3. reviewTime can be use to develop temporary models. 
4. It is possible to do a gender discrimination using reviewerName. Nevertheless, the format of this variable is very varied and we don't trust its quality, so we will drop it. 
5. summary is an abstract of reviewText. We will drop summary as reviewText gives more information. 

Based on point 2 we can study how many reviews are verified. If the percentage of verified reviews is high we can drop the not verified reviews and reduce the amount of data and improve the data quality. 

In [13]:
print('Percentage of verified reviews:', reviews['verified'].loc[reviews['verified']==True].count()/len(reviews['verified'])*100,'%')

Percentage of verified reviews: 93.78099033214843 %


### Reviews precleaning
Following these decisions we can clean the Reviews data frame.

In [14]:
def reviews_precleaning(data):
    data = data.drop(['reviewerName','summary'], axis = 1)
    data = data.drop(data[data['verified'] == False].index)
    return data

reviews = reviews_precleaning(reviews)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 828405 entries, 0 to 883339
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      828405 non-null  int64  
 1   overall         828405 non-null  float64
 2   verified        828405 non-null  bool   
 3   reviewTime      828405 non-null  object 
 4   reviewerID      828405 non-null  object 
 5   asin            828405 non-null  object 
 6   reviewText      827233 non-null  object 
 7   unixReviewTime  828405 non-null  int64  
 8   vote            72359 non-null   float64
 9   style           288980 non-null  object 
 10  image           24104 non-null   object 
dtypes: bool(1), float64(2), int64(2), object(6)
memory usage: 70.3+ MB


### Data quality exploration

Now we will perform an exploration on the variables of the data frames to get a better understanding of them. 

First of all, we will inspect the brand distribution of the products:
- We can se that there are popular brands. This will be helpfull for the NLP model. 


In [15]:
print('--------- Popular brands and Nº of products ------------')
meta.groupby('brand')['asin'].count().sort_values(ascending = False)

--------- Popular brands and Nº of products ------------


brand
ZANZEA                   1108
PERSUN                    900
AmeriMark                 864
Allegra K                 777
OURS                      592
                         ... 
Pressman                    1
Final Fight                 1
Final Touch Gifts           1
Premium Life Sankuwen       1
" "                         1
Name: asin, Length: 18513, dtype: int64

We can also study the how many time each user has done a review.
- As we can see, the maximum number of reviews per user is 24. 
- If we study how many users have done n reviews with n=1,2,...,24 we can see a drawback for the recommender model. Around 650k users have done only 1 review while only less than 200 users have done more than 10 reviews. 

In [16]:
print('--------- How many reviews has made each user ------------')
reviews.groupby('reviewerID')['asin'].count().sort_values(ascending=False)

--------- How many reviews has made each user ------------


reviewerID
AXRC1EH0U2WZ9     24
A3J5XBZ9ZNEZOC    21
AKW4JIYF5I08K     19
A4455JHTQFWAO     19
A2VHVODX6A92OA    19
                  ..
A2CDMAH77SVPKY     1
A2CDMNYZWU7WGN     1
A2CDMX8OQESVON     1
A2CDN3CISIZLZ1     1
AZZZU2YUCMUUW      1
Name: asin, Length: 706897, dtype: int64

In [17]:
print('--------- How many users have made n reviews ------------')
clientes_reviews = pd.DataFrame(reviews.groupby('reviewerID')['asin'].count().sort_values(ascending=False))
clientes_reviews.rename(columns = {'asin':'reviews'},inplace = True)
pd.DataFrame(clientes_reviews.value_counts()).sort_values('reviews')


--------- How many users have made n reviews ------------


,0
reviews,
1,620626
2,66342
3,12124
4,4664
5,1429
6,631
7,315
8,475
9,121


Finally, we'll see how many reviews has each product and its mean:
- We can see the same trend as we did with the users reviews. There are products with a high number of reviews and a majority of products with very few, as we can see thanks to the mean number of reviews.
- We will only use a certain percentage of products with the highest number of reviews. Moreover, we will introduce a Weighted Rating. This will consider the number of reviews to calculate the ranking of the products.

In [18]:
print('--------- How many reviews has each product and mean overall ------------')
products_count = reviews.groupby('asin', as_index=False)['overall'].count()
products_count.rename(columns={'overall':'n_reviews'},inplace = True)
products_mean = reviews.groupby('asin', as_index=False)['overall'].mean()
products_mean.rename(columns={'overall':'mean_overall'},inplace = True)

products_mean = products_mean.merge(products_count, on = 'asin')
products_mean.sort_values('n_reviews',ascending = False)

--------- How many reviews has each product and mean overall ------------


,asin,mean_overall,n_reviews
2377,B000V0IBDM,4.515074,4279
1718,B000KPIHQ4,4.517330,4270
34516,B00I0VHS10,4.428215,3608
67606,B00RLSCLJM,4.826670,3577
2596,B000YFSR5G,3.811983,2420
...,...,...,...
108923,B014X4ZHA6,4.000000,1
108921,B014X4ONCY,5.000000,1
108920,B014X34988,5.000000,1
108919,B014X3497E,3.000000,1


In [19]:
print('Mean number of reviews')
print(products_mean['n_reviews'].mean())

Mean number of reviews
4.618800702517354


# Simple recommender based on score

The first component on the recommender model will be the Weighted Rating introduced before. 
- Weighted Rating (WR) = (v/(v+m)*R)+(m/(v+m)*C)

    - v is the number of votes for the movie   
    - m is the minimum votes required to be listed in the chart
    - R is the average rating of the movie
    - C is the mean vote across the whole report
    
- The next step is to determine an appropriate value for m, the minimum votes required to be listed in the chart. We will use 80th percentile as our cutoff. In other words, for a product to feature in the charts, it must have more revies than at least 80% of the products in the list.

In [20]:
WR_count = reviews.groupby('asin', as_index=False)['overall'].count()
WR_count.rename(columns={'overall':'n_reviews'},inplace = True)
WR = reviews.groupby('asin', as_index=False)['overall'].mean()
WR.rename(columns={'overall':'mean_overall'},inplace = True)
WR = WR.merge(WR_count, on = 'asin')
C = WR['mean_overall'].mean()
m = WR['n_reviews'].quantile(0.90)
WR = WR.loc[WR['n_reviews']>=m]

def weighted_rating(data,m=m,C=C):
    v = data['n_reviews']
    R = data['mean_overall']
    
    return (v/(v+m) * R) + (m/(m+v) * C)

WR['score'] = WR.apply(weighted_rating, axis=1)

In [21]:
#Join the WR df to the meta df
meta = meta.merge(WR, on = 'asin')
meta.sort_values('score', ascending = False).head(50)
# This new meta df only has products with a number of review higher than m

,Unnamed: 0,title,brand,feature,rank,date,asin,mean_overall,n_reviews,score
12678,110719,Dasom Womens Fashion Socks,Dasom,"['Made In Korea', ""One Size Fits All: Women's ...","151,477inClothing,ShoesJewelry(",5 star,B0148B7EJ6,4.900763,131,4.847084
29,977,BOX1MM Nickel Free Italian Sterling Silver 1mm...,NaN,NaN,"641,202inClothing,Shoesamp;Jewelry(",5 star,B0006HB4XE,4.883436,163,4.840574
15998,145731,Working Class Kid's Lab Coat Durable Lab Coats...,Working Class,['MAKE SCIENCE FUN: Science kits and science e...,"10,120inClothing,ShoesJewelry(",5 star,B01B5BWTNS,4.839104,491,4.825096
8684,71820,MJ Metals Jewelry 2mm to 10mm White Tungsten C...,MJ Metals Jewelry,['Handsome Band - Crafted with beautiful tungs...,"6,926inClothing,ShoesJewelry(",MJ Metals Jewelry,B00RLSCLJM,4.826670,3577,4.824748
12757,111284,Labor Delivery Push Hospital Non Skid Heart So...,Baby Be Mine,"['Wow the ward in these one of a kind socks!',...","85,489inClothing,ShoesJewelry(",5 star,B014EB2ADA,4.909091,77,4.820210
11104,95584,Sad Pepe Lapel Pin,NaN,['Package Dimensions:\n \n2...,"691,843inClothing,ShoesJewelry(",MP-02SP,B00YWSNUO4,4.887755,98,4.818072
2749,25301,Stephen Joseph Quilted Purse,NaN,"[""FUN & FUNCTIONAL: Colorful and cute characte...","14,832inClothing,ShoesJewelry(",5 star,B00D8FJF6O,4.828418,373,4.810257
11929,103025,Oberon Design Tree of Life Embossed Genuine Le...,Oberon Design,"['100% Leather', 'Made in USA', 'HANDCRAFTED I...","258,660inClothing,ShoesJewelry(",5 star,B011PLYVCA,4.854305,151,4.809479
11921,102991,Oberon Design Bold Celtic Embossed Genuine Lea...,Oberon Design,"['100% Leather', 'Made in USA', 'HANDCRAFTED I...","109,542inClothing,ShoesJewelry(",5 star,B011P3DFSE,4.916667,60,4.804442
3794,32510,MJ Metals Jewelry 2mm to 10mm White Tungsten C...,MJ Metals Jewelry,['Handsome Band - Crafted with beautiful tungs...,"8,871inClothing,ShoesJewelry(",MJ Metals Jewelry,B00G8Q7JZ4,4.812995,631,4.802347


In [22]:
print('Mean number of reviews:',meta['n_reviews'].mean())
print('Lower number of reviews:',meta['n_reviews'].min())

Mean number of reviews: 28.495569653436796
Lower number of reviews: 7


As result of this process we have added 3 new variables to the Meta data frame: 
- mean_overall: mean of the ratings
- n_reviews: number of reviews
- score: weighted rating

# NLP model
The NLP part of the recommender model will use the information given in the title, brand, feature and rank of the Meta data frame to find similar products with high score of any product in the data frame. Besides, it will use the information on the reviewText of the Reviews data frame to also find similar products. Both lists of similar items will be use in machine learning models to do the final recomendations of product given a user. 

The NLP part will require data cleaning and transformation for each variable:



- To do the processing of the reviewText variable we will perform an inspection on all the reviews of one product. Later we will make a transformation to unite all the reviews. 

In [23]:

pd.options.display.max_colwidth = 5000
reviews['reviewText'] = reviews['reviewText'].fillna('')
reviews = reviews.dropna(subset = ['asin'], axis = 0)
reviews[['asin','reviewText']][reviews['asin'] == '7106116521'].head(50)


,asin,reviewText
0,7106116521,Exactly what I needed.
1,7106116521,"I agree with the other review, the opening is too small. I almost bent the hook on some very expensive earrings trying to get these up higher than just the end so they're not seen. Would not buy again but for the price, not sending back."
3,7106116521,too tiny an opening
5,7106116521,Exactly what I wanted.
6,7106116521,"These little plastic backs work great. No more loosing ""hook"" earrings! Wish I had ordered these sooner before I had lost some of my favorite earrings!"
396912,7106116521,Diameter is too small for earrings
396913,7106116521,"Now I have ba Kings for all of my costume jewelry. I am very satisfied with the quality, the price and the shipping."
396914,7106116521,"I am always losing the backs to earrings, so I will always have replacements now. Love them!!!"
396915,7106116521,"not happy,don't fit earrings, some don't even have holes,waste of money"
396916,7106116521,Perfect for your earrings. I use all the time so I don't loose my expensive earings


In [24]:
reviews['full_text'] = reviews[['asin','reviewText']].groupby('asin')['reviewText'].transform(lambda x: ''.join(x))
pd.options.display.max_colwidth = 5000
reviews_asin = reviews[['asin','full_text']].drop_duplicates(subset = ['asin'])

In [25]:
meta = meta.merge(reviews_asin, on='asin',how='left')
meta.head(2)

Unnamed: 0  \
0         10   
1         69   

                                                                        title  \
0  Milliongadgets(TM) Earring Safety Backs For Fish Hook Small Earrings (150)   
1                                        Buxton Heiress Pik-Me-Up Framed Case   

    brand  \
0     NaN   
1  Buxton   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        feature  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           NaN   
1  ['Leather', 'Imported', 'synthetic lining', 'Flap closure', 'Authentic Crunch Leather', 'Rich Floral Embossed Logo', 'Goldtone Hardware', 'Large Snap Pocket', 'You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused condition.', 'Read the full returns policy', 'Go to Your Orders to start the return', 'Print the return shipping label', 'Ship it!', 'Product Dimensions:\n                    \n5 x 6 x 2 inches', 'Shipping Weight:\n                    \n0.8 ounces (View shipping rates and policies)']   

                                 rank      date        asin  mean_overall  \
0   2,081,098inClothing,ShoesJewelry(  fishhook  7106116521      3.866667   
1  43,930inClothing,Shoesamp;Jewelry(    5 star  B00007GDFV      3.642105   

   n_reviews     score  \
0         15  3.858982   
1         95  3.655859   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

Now we use the function clean_full_text to work on the full_text variable removing useless characters and changing the format for future steps. 

In [26]:

def clean_full_text(data):
    data['cleaned_full_text'] = data['full_text'].replace([';',',','-','\(','\)','/','&','%',':','[0-9]','\#','\.','"','\'','\\n','\!','\?'],'',regex=True)
    data['cleaned_full_text'] = data['cleaned_full_text'].replace('  ',' ', regex=True)
    data['cleaned_full_text'] = data['cleaned_full_text'].fillna('')
    data['cleaned_full_text'] = data['cleaned_full_text'].str.lower()
    data['cleaned_full_text'] = data['cleaned_full_text'].str.split(' ')
    return data

clean_full_text(meta)
pd.options.display.max_colwidth = 1000
meta[['brand','full_text','cleaned_full_text']].head(2)

,brand,full_text,cleaned_full_text
0,NaN,"Exactly what I needed.I agree with the other review, the opening is too small. I almost bent the hook on some very expensive earrings trying to get these up higher than just the end so they're not seen. Would not buy again but for the price, not sending back.too tiny an openingExactly what I wanted.These little plastic backs work great. No more loosing ""hook"" earrings! Wish I had ordered these sooner before I had lost some of my favorite earrings!Diameter is too small for earringsNow I have ba Kings for all of my costume jewelry. I am very satisfied with the quality, the price and the shipping.I am always losing the backs to earrings, so I will always have replacements now. Love them!!!not happy,don't fit earrings, some don't even have holes,waste of moneyPerfect for your earrings. I use all the time so I don't loose my expensive earingsever helpfulYou get exactly what is described.Good quality, would order from seller again.Perfect for lost backsOK, so they are a little tigh...","[exactly, what, i, neededi, agree, with, the, other, review, the, opening, is, too, small, i, almost, bent, the, hook, on, some, very, expensive, earrings, trying, to, get, these, up, higher, than, just, the, end, so, theyre, not, seen, would, not, buy, again, but, for, the, price, not, sending, backtoo, tiny, an, openingexactly, what, i, wantedthese, little, plastic, backs, work, great, no, more, loosing, hook, earrings, wish, i, had, ordered, these, sooner, before, i, had, lost, some, of, my, favorite, earringsdiameter, is, too, small, for, earringsnow, i, have, ba, kings, for, all, of, my, costume, jewelry, i, am, very, satisfied, with, ...]"
1,Buxton,"mother - in - law wanted it as a present for her sister. she liked it and said it would work.Item is of good quality. Looks great, too. But it does not fit 100's! But it can be stretched to fit them if you carefully push bottom of case with your fingers. Then shove in a pack of 100's(max) and level it out as you close case stretching case closed. Leave a pk.in it for month or so & was la! Buxton is usually a good quality product!I had used my last el-cheapo fake leather cigarette case for SEVEN YEARS. It still closed completely but the plastic made to look like leather was literally falling off, so it was time for a new one. Cigarette cases for kings size cigs are not easy to come by these days I discovered, but I was thrilled to find this one on Amazon. It was a great price, REAL LEATHER, and even had the cool zipper pouch on the back. I was so excited to get my case and toss that other one! Well, within THREE DAYS one of the gold clasps literally broke off! I couldn't believe it!...","[mother, in, law, wanted, it, as, a, present, for, her, sister, she, liked, it, and, said, it, would, workitem, is, of, good, quality, looks, great, too, but, it, does, not, fit, s, but, it, can, be, stretched, to, fit, them, if, you, carefully, push, bottom, of, case, with, your, fingers, then, shove, in, a, pack, of, smax, and, level, it, out, as, you, close, case, stretching, case, closed, leave, a, pkin, it, for, month, or, so, was, la, buxton, is, usually, a, good, quality, producti, had, used, my, last, elcheapo, fake, leather, cigarette, case, for, seven, years, it, still, closed, ...]"


- Now we will repeat the process for the variables in the Meta data frame

In [27]:

ranks = pd.Series(meta['rank'])
def clean_rank(data):
    data['cleaned_rank'] = ranks.str.extract('([A-Z].*)',expand=False)
    data['cleaned_rank'] = data['cleaned_rank'].replace([';',',','\(','%'],'',regex=True)
    data['cleaned_rank'] = data['cleaned_rank'].fillna('')
    data['cleaned_rank'] = data['cleaned_rank'].str.split(r'([A-Z][^A-Z]*)')
    
    return data

clean_rank(meta)
pd.options.display.max_colwidth = 1000
meta[['rank','cleaned_rank']].head(5)


,rank,cleaned_rank
0,"2,081,098inClothing,ShoesJewelry(","[, Clothing, , Shoes, , Jewelry, ]"
1,"43,930inClothing,Shoesamp;Jewelry(","[, Clothing, , Shoesamp, , Jewelry, ]"
2,NaN,[]
3,"6,123,960inClothing,Shoesamp;Jewelry(","[, Clothing, , Shoesamp, , Jewelry, ]"
4,"4,785,781inClothing,ShoesJewelry(","[, Clothing, , Shoes, , Jewelry, ]"


In [28]:
title = pd.Series(meta['title'])
def clean_title(data):
    data['cleaned_title'] = data['title'].replace([';',',','-','\(','\)','/','&','%',':','[0-9]','\#','\.','"','\''],'',regex=True)
    data['cleaned_title'] = data['cleaned_title'].fillna('')    
    data['cleaned_title'] = data['cleaned_title'].str.lower()
  
    return data

clean_title(meta)
pd.options.display.max_colwidth = 1000
meta[['brand','title','cleaned_title']].head(5)

,brand,title,cleaned_title
0,NaN,Milliongadgets(TM) Earring Safety Backs For Fish Hook Small Earrings (150),milliongadgetstm earring safety backs for fish hook small earrings
1,Buxton,Buxton Heiress Pik-Me-Up Framed Case,buxton heiress pikmeup framed case
2,NaN,Jockey Women's Underwear Elance String Bikini,jockey womens underwear elance string bikini
3,NaN,Paul Fredrick Men's Pinpoint Snap Tab Collar Button Cuff Dress Shirt White 17.0/33,paul fredrick mens pinpoint snap tab collar button cuff dress shirt white
4,NaN,Paul Fredrick Men's Pinpoint Snap Tab Collar French Cuff Dress Shirt White 17.0/34,paul fredrick mens pinpoint snap tab collar french cuff dress shirt white


In [29]:

def clean_feature(data,max_length):
    data['cleaned_feature'] = data['feature'].replace(['  ','\'','\[','\]','\.','[0-9]','\:','%','"','/'],'',regex=True)
    data['cleaned_feature'] = data['cleaned_feature'].fillna('')
    prueba = data['cleaned_feature'].str.split(',')
    
    for ii in range(0,len(prueba)):
        for jj in range(0,len(prueba[ii])):
            
            if prueba[ii][jj].startswith('Package Dimensions') == True:
                prueba[ii][jj] = ''
            
            elif prueba[ii][jj].startswith(' Package Dimensions') == True:
                prueba[ii][jj] = ''
            
            elif prueba[ii][jj].startswith(' Product Dimensions') == True:
                prueba[ii][jj] = ''
            
            elif prueba[ii][jj].startswith(' Ship it') == True:
                prueba[ii][jj] = ''
            
            elif prueba[ii][jj].startswith(' Shipping') == True:
                prueba[ii][jj] = ''
                
            elif prueba[ii][jj].startswith(' Print the return') == True:
                prueba[ii][jj] = ''
                
            elif prueba[ii][jj].startswith(' Go to Your Orders') == True:
                prueba[ii][jj] = ''
            
            elif prueba[ii][jj].startswith(' Read the full returns policy') == True:
                prueba[ii][jj] = ''
                
            elif len(prueba[ii][jj]) > max_length:
                prueba[ii][jj] = ''
                       
    data['cleaned_feature'] = prueba.str.join(',')
    data['cleaned_feature'] = data['cleaned_feature'].replace(',',' ', regex=True)
    data['cleaned_feature'] = data['cleaned_feature'].str.split(' ')

    return data

clean_feature(meta,50)
pd.options.display.max_colwidth = 1000
meta[~meta['feature'].isna()][['feature','cleaned_feature']].head(5)

,feature,cleaned_feature
1,"['Leather', 'Imported', 'synthetic lining', 'Flap closure', 'Authentic Crunch Leather', 'Rich Floral Embossed Logo', 'Goldtone Hardware', 'Large Snap Pocket', 'You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused condition.', 'Read the full returns policy', 'Go to Your Orders to start the return', 'Print the return shipping label', 'Ship it!', 'Product Dimensions:\n \n5 x 6 x 2 inches', 'Shipping Weight:\n \n0.8 ounces (View shipping rates and policies)']","[Leather, , Imported, , synthetic, lining, , Flap, closure, , Authentic, Crunch, Leather, , Rich, Floral, Embossed, Logo, , Goldtone, Hardware, , Large, Snap, Pocket, , , , , , , ]"
3,"['Package Dimensions:\n \n15 x 8.3 x 2.4 inches', 'Shipping Weight:\n \n12 ounces']","[, ]"
4,"['Package Dimensions:\n \n15.4 x 8.8 x 1.9 inches', 'Shipping Weight:\n \n13.6 ounces']","[, ]"
6,"['Package Dimensions:\n \n14.8 x 8.9 x 1 inches', 'Shipping Weight:\n \n0.8 ounces']","[, ]"
7,"['Package Dimensions:\n \n15.3 x 9.6 x 1.8 inches', 'Shipping Weight:\n \n11.2 ounces']","[, ]"


Now we have cleaned all the variables that we'll use in the NLP model. Next step will be performing a final cleaning for the variables at the same time that we apply a stemmer. 
- Stemming is the process of producing morphological variants of a root/base word. A stemming algorithm reduces the words “chocolates”, “chocolatey”, “choco” to the root word, “chocolate”. 

Finally, we will join all the Meta variables into the soup variable, that will be used in the function get_recommendations(). 

In [30]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        

In [31]:
features = ['cleaned_rank','cleaned_feature','cleaned_title','brand']

In [32]:
features = ['cleaned_rank','cleaned_feature','brand','cleaned_title','cleaned_full_text']
meta['cleaned_title'] = meta['cleaned_title'].astype(str).str.split()

for feature in features:
    if feature == 'brand':
        meta[feature] = meta[feature].apply(clean_data)
    else:
        aux = meta[feature].apply(lambda x:[stemmer.stem(i) for i in x] )
        meta[feature] = aux.apply(clean_data)
        del aux
    
meta[features].head(5)

,cleaned_rank,cleaned_feature,brand,cleaned_title,cleaned_full_text
0,"[, cloth, , shoe, , jewelri, ]",[],,"[milliongadgetstm, earring, safeti, back, for, fish, hook, small, earring]","[exact, what, i, neededi, agre, with, the, other, review, the, open, is, too, small, i, almost, bent, the, hook, on, some, veri, expens, earring, tri, to, get, these, up, higher, than, just, the, end, so, theyr, not, seen, would, not, buy, again, but, for, the, price, not, send, backtoo, tini, an, openingexact, what, i, wantedthes, littl, plastic, back, work, great, no, more, loos, hook, earring, wish, i, had, order, these, sooner, befor, i, had, lost, some, of, my, favorit, earringsdiamet, is, too, small, for, earringsnow, i, have, ba, king, for, all, of, my, costum, jewelri, i, am, veri, satisfi, with, ...]"
1,"[, cloth, , shoesamp, , jewelri, ]","[leather, , import, , synthet, line, , flap, closur, , authent, crunch, leather, , rich, floral, emboss, logo, , goldton, hardwar, , larg, snap, pocket, , , , , , , ]",buxton,"[buxton, heiress, pikmeup, frame, case]","[mother, in, law, want, it, as, a, present, for, her, sister, she, like, it, and, said, it, would, workitem, is, of, good, qualiti, look, great, too, but, it, doe, not, fit, s, but, it, can, be, stretch, to, fit, them, if, you, care, push, bottom, of, case, with, your, finger, then, shove, in, a, pack, of, smax, and, level, it, out, as, you, close, case, stretch, case, close, leav, a, pkin, it, for, month, or, so, was, la, buxton, is, usual, a, good, qualiti, producti, had, use, my, last, elcheapo, fake, leather, cigarett, case, for, seven, year, it, still, close, ...]"
2,[],[],,"[jockey, women, underwear, elanc, string, bikini]","[i, like, cotton, and, i, use, to, buy, costco, cotton, one, but, i, feel, there, is, some, synthet, mixtur, and, i, hate, thati, tri, other, product, but, other, product, are, not, as, good, as, this, this, is, my, rd, time, to, order, this, idemso, far, this, is, the, best, cotton, underwear, which, has, a, good, shape, and, colorsi, just, order, more, i, bought, some, at, kohl, and, love, them, i, am, tire, of, lacey, underwear, fall, apart, when, i, found, these, on, amazon, i, order, more, and, was, just, as, pleas, they, have, full, coverag, and, a, littl, bit, of, ...]"
3,"[, cloth, , shoesamp, , jewelri, ]","[, ]",,"[paul, fredrick, men, pinpoint, snap, tab, collar, button, cuff, dress, shirt, white]","[great, shirti, like, the, shirt, they, fit, as, expect, howev, they, will, requir, a, littl, more, mainten, in, that, they, are, not, wrinkl, freeexcellentgreatcuff, are, a, littl, tight, tab, collar, doesnt, fit, tie, veri, well, it, leav, open, on, the, side, after, tri, differ, knot, sizesi, was, look, for, a, tab, collar, dress, shirt, i, have, alway, like, that, style, but, late, i, had, been, unabl, to, find, ani, i, look, on, amazon, and, lo, and, behold, there, they, were, reason, price, fast, deliveri, and, excel, qualiti, i, have, absolut, no, complaint, and, i, intend, to, purchas, sever, more, in, ...]"
4,"[, cloth, , shoe, , jewelri, ]","[, ]",,"[paul, fredrick, men, pinpoint, snap, tab, collar, french, cuff, dress, shirt, white]","[if, you, want, a, look, that, take, it, back, to, the, old, school, buy, this, shirt, it, isnt, as, appreci, as, it, use, to, be, but, it, just, as, relev, now, as, it, was, back, then, perfect, with, a, classic, black, tie, for, job, interview, howev, stay, awar, of, the, button, lead, up, to, the, cufflink, sometim, they, can, snap, off, if, not, attend, to, other, than, that, a, shirt, of, sophist, guaranteedlov, the, shirt, my, husband, love, it, so, much, he, love, the, neck, and, the, materi, bar, for, the, tie, fit, great, i, recommend, this, product, your, husband, ...]"


In [33]:
def create_soup(x):
    return ' '.join(x['cleaned_rank']) + ' ' + ' '.join(x['cleaned_feature']) + ' ' + ' '.join(x['cleaned_title']) + ' ' + x['brand'] #+ ' ' + ' '.join(x['cleaned_full_text'])


In [34]:
meta['soup'] = meta.apply(create_soup, axis=1)
pd.options.display.max_colwidth = 500
meta[['soup','asin']].head(5)

,soup,asin
0,cloth shoe jewelri milliongadgetstm earring safeti back for fish hook small earring,7106116521
1,cloth shoesamp jewelri leather import synthet line flap closur authent crunch leather rich floral emboss logo goldton hardwar larg snap pocket buxton heiress pikmeup frame case buxton,B00007GDFV
2,jockey women underwear elanc string bikini,B00008ID39
3,cloth shoesamp jewelri paul fredrick men pinpoint snap tab collar button cuff dress shirt white,B00008JPRZ
4,cloth shoe jewelri paul fredrick men pinpoint snap tab collar french cuff dress shirt white,B00008JVTT


In [35]:
meta['reviews']=meta['cleaned_full_text'].apply(lambda x: ' '.join(x))

Now we'll compute Term Frequency-Inverse Document Frequency (TF-IDF) vectors for each product. The Term Frequency is the relative frequency of a word given as (term instances)/(total instances). Inverse Document Frequency is the relative count of documents containing the term is given as log(number of documents/documents with term). This will give us a matrix where each column represents a word in the overview data frame and each row represents a product. This is done to reduce the importance of words that occur frequently in the soup. 

We also use stop_words='english' to eliminate all words that don't give us information such as: in, at, for, a ...

We will be using the cosine similarity to calculate a numeric quantity that denotes the similarity between two products. Eventually, calculating the dot product will directly give us the cosine similarity score. 

In [36]:
tfidf = TfidfVectorizer(stop_words='english')

def cosin_similarity_meta(data,variable):
    tfidf_matrix = tfidf.fit_transform(data[variable])
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    return cosine_sim

cosine_soup = cosin_similarity_meta(meta,'soup')

In the reviews variable, the frecuency of a given word is important becouse it can reflex a popular opicion. Therefore, we will use Count Vectorizerinstead of TF-IDF. 

In [37]:
count = CountVectorizer(stop_words='english')

def cosin_similarity_reviews(data,variable):
    tfidf_matrix = count.fit_transform(data[variable])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

cosine_reviews = cosin_similarity_reviews(meta,'reviews')

Finally, we can build the get_recomendations() function to obtain similar products given another product. The imputs of this functions are:

- product: 'asin code of the product. 
- num_of_recom: number of recommendations. 
- cosine_sim: cosine similarity index between the products. It can be cosine_soup or cosine_reviews. 
- variable_similarity: variable with which to compare. It can be soup or reviews.

In [38]:
indices = pd.Series(meta.index, index=meta['asin']).drop_duplicates()
def get_recommendations(product, num_of_recom, cosine_sim , variable_similarity):
    # Get the index of the product that matches the asin
    idx = indices[product]
    
    # Get the pairwsie similarity scores of all products with that product
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the products based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the n=num_of_recom most similar products
    sim_scores = sim_scores[1:num_of_recom +1]
    sim_scores_np = np.array(sim_scores)
    sim_scores_np = sim_scores_np[:,1]
    
    # Get the product indices
    product_indices = [i[0] for i in sim_scores]

    # Create the recommendations DataFrame adding the similarity score
    recommendations = pd.DataFrame(meta[['asin','title',variable_similarity,'score']].iloc[product_indices])
    new_variable =  'nlp_similarity'
    recommendations[new_variable] = sim_scores_np.tolist()
          
    return recommendations


In [39]:
test_product = 'B0148B7EJ6'
print('Producto:') 
print(meta['title'].loc[meta['asin'] == test_product])
get_recommendations(test_product,50,cosine_soup,'soup')

Producto:
12678    Dasom Womens Fashion Socks
Name: title, dtype: object


,asin,title,soup,score,nlp_similarity
14506,B017RPGLIU,"Tipi Toe Women's 12-Pair Ragg Cotton Crew Boot Socks, BT09-12B (Fits shoe 6-9, sock size 9-11), Assorted Colors",cloth shoe jewelri materi cotton polyest acryl spandex size fit shoe size - (sock size -) slight longer than crew length tipi toe women pair ragg cotton crew boot sock btb fit shoe sock size assort color tipitoe,3.998933,0.213978
6886,B00N3IZENO,Jefferies Socks Big Girls' Eyelet Lace Socks (Pack of 3),cloth shoe jewelri cotton nylon spandex import machin wash this is a pair pack of lace sock lace trim on a seamless toe turn cuff reciproc heel and toe with spandex for fit see size chart for detail size inform jefferi sock big girl eyelet lace sock pack of,4.592800,0.198435
16461,B01CEZSSXY,Blue Q Women's Novelty Ankle Socks - You Smell Delicious (Women's Size 5-10),cloth shoe jewelri comb cotton nylon spandex one size fit women shoe size - machin wash cold tumbl dri medium heat do not bleach blue q women novelti ankl sock you smell delici women size blueq,4.775181,0.180048
15886,B01AWLLDFC,Disney Princess Socks Size 4-6 (7 Pairs) Shoe Size 7-10,cloth shoe jewelri disney princess sock size pair shoe size,3.921258,0.179981
10480,B00X23GJMK,Yoga Sprout Baby No Show Socks,cloth shoe jewelri cotton nylon spandex import pull on closur machin wash pack of no show sock stretchabl cotton or easi fit rise to ankl for a no show appear soft and comfort yoga sprout babi no show sock,4.300841,0.177784
17484,B01E4FJG9W,Yoga Sprout Baby No Show Socks,cloth shoe jewelri cotton nylon spandex import pull on closur machin wash pack of no show sock stretchabl cotton or easi fit rise to ankl for a no show appear soft and comfort yoga sprout babi no show sock,4.341297,0.177784
9991,B00VNCJKVY,"Women's Floral Summer Pajama Short Set, Blue, Large, Machine Washable",cloth shoe jewelri women floral summer pajama short set blue larg machin washabl collectionsetc,4.328458,0.170851
15428,B019VW1QL4,Fantastic Cos Colorful Womens Mens Crazy Fun Casual Cotton Socks,cloth shoe jewelri the fit size for us men and women - packag pair halloween bat style sock the materi is made of cotton polyest spandex moon bat pumpkin fantast cos color women men crazi fun casual cotton sock fantasticcos,4.288095,0.167600
7924,B00P81WNQ0,3 Pairs Men's Knee-High Dress Socks,cloth shoe jewelri comb cotton nylon spandex materi comb cotton nylon spandex knee-high sock the smooth light sock premium comb cotton feel ultra-soft insid and out pair men kneehigh dress sock y2y2,4.072178,0.167124
17696,B01EICRJ18,3 Pairs Men's Knee-High Dress Socks,cloth shoe jewelri comb cotton nylon spandex materi comb cotton nylon spandex knee-high sock the smooth light sock premium comb cotton feel ultra-soft insid and out pair men kneehigh dress sock y2y2,3.662874,0.167124


In [40]:
test_product = 'B0148B7EJ6'
print('Producto:') 
print(meta['title'].loc[meta['asin'] == test_product])
get_recommendations(test_product,50,cosine_reviews,'reviews')

Producto:
12678    Dasom Womens Fashion Socks
Name: title, dtype: object


,asin,title,reviews,score,nlp_similarity
16461,B01CEZSSXY,Blue Q Women's Novelty Ankle Socks - You Smell Delicious (Women's Size 5-10),i bought these as a gift for a friend and she love them they are super snazzi i havent worn them but they seem to be pretti good qualiti the price was averag this brand has great stuff ive bought blue q sock for a few peopl and they all love themlov these socksthes are cute sock nice and soft if you have a big anklelow calf the top band might be a littl tight for averag leg this wont be an issuethes are the bombwel maderaisin helleverywher i go now they just need to make one that say ball bu...,4.775181,0.840025
13657,B01668UZ2W,Socksense Totoro Women's Socks 3pairs(3color)=1pack Made in Korea,just perfectso cute i love korean fashion got these for my nieceveri cute love iti got these for my friend who is obsess with studio ghibli as am i and she love them she wear the grey one the most becaus they match her shoe which she wear everi day but she wear all of them and she has told me that they are veri nice to sleep in dure the winter i might get them for myself somedaygreat veri cutesup cutethes sock are so ador and fit perfect they arent too thin and did not shrink in the dryer i ...,4.670866,0.802371
15747,B01AL61AF6,"Dani's Choice Casual Cotton Print Socks, Penguin 5 Pairs, One Size",i was hope to use these as cycl sock theyr pretti thin but realli cute i love these sock they fit perfect too im a size in women everyon compliment themthes look awesom ive ad them to my evergrow collect of realli cool sock my friend alway get a kick out of thembeauti aveng sock the pattern and color were exact as describedlov themthes sock are incred ador so amaz and so worth it i just wish there was a hawkey pairwhen i purchas these sock i had search for superhero sock for boy the product ...,4.691968,0.801461
3153,B00ECGFAIA,"Sock It To Me Zebra Women's Knee High Socks,size 5-10",these sock are fabul the design is so strike and i find the construct of the sock to be excel onli two comment that brought the rate down to a fourstar from a five while the bodi of the sock fit well the sock are a bit tight at the top i dont mind a snug fit but i do find i want to take them off in the middl of the night if i go to sleep with them on and i didnt realiz that there would be so mani thread on the insid of the sock yes the design is intric but i guess i figur part of the design...,4.395200,0.781488
6430,B00M21O9L6,"Bride of Frankenstein Knee High Tube Socks (Black/Purple, 5-10)",im a guy i love knee high sock for runningi get sock it to me sock all the timeso far i love them they are pretti durabl and stay upif they can work with me they can work for anyonelov itfast deliveri look just like the photo hope they wash well look forward to wear sock with the purpl wig on vike nfl game day in octobermi friend love this so much great qualiti if you wanna spend the moneybest sock everlooov these i have a women sz foot and was worri these may not fit but theyr actual a litt...,4.766605,0.781057
3150,B00ECFYWEE,Sock It To Me Women's Knee High Socks Dandelion Grey,i think these are realli cute sock and are great to wear under tall boot i have size feet and these seem to fit fine not too snugtrendi fit is perfect great for self or as a gift true fit sock stay pull up do not fall downthes sock ship out to me fast less than two week they fit great they are not too thick but also not thin in a way which they would be consid cheap perfect to wear on a fall day under boot or jean also i am tall and they go all the way up to my knee and fit my feet size no p...,4.717760,0.773694
2309,B00BSE5S9A,Sock It To Me Retro Owl Knee High Socks,these are so fun they look like candi corn and have owl on them i am a second grade teacher and everyon love them especi me these are super cute but it you have calv the owl isnt stretchi enough i gave them to my yr old daughter sinc her leg are scrawnynic cute socksi order these sock for my daughterinlaw who

# Colaborative Filtering

Collaborative Filtering consists on getting a prediction based on the taste of what are considered similiar users. It is, therefore, based on the idea that the closer the reviews that two clients have given to certain products are the more likely it is that their reviews in an other product is similar. Hence, for the prediction of the review of a certain product, the reviews of other clients that have given similar reviews to other products are used.

For this part, two different groups of algorithms will be used. Firstly, a series of algorithms based on Matrix Factorization will be trained and tested followed by a number of models based on the K-nearest Neighbors algorithm.

The folowing used are the following:

- Single Value Descomposition (SVD)
- SVD taking into account implicit rankings (SVDpp)
- Non-negative Matrix Factorization (NMF)
- Slope One
- Co Clustering
- Basic KNN (KNNBAsic)
- Centered KNN (KNNwithMeans)
- KNN taking into consideration the Z-score normalization of each user (KNNwithZscore)
- KNN taking considering a baseline rating (KNNBaseline)


As evaluation method, RMSE is selected to compare the results of the models.

Before testing the models,a serie of transformations will be applied to the dataset for the developing of the model. For example, since it is important to predict the model that the clientes taken into consideration have some products in common, clients with less than 3 purchases, as well as products with less than 3 reviews, will not be taken into consideration. This also helps to reduce the dimensionality of the obtained matrices for the models.

In [41]:
#Previous transformations
reader = Reader()
reviewsR = reviews.merge(WR, on = 'asin')

def _filter_by_count(elements, min_count):

    unique_elements, element_counts = np.unique(elements,
                                                return_counts=True)

    return unique_elements[element_counts >= min_count]

min_user_interactions=3
min_item_interactions=3

rev = reviewsR[["reviewerID", "asin", "overall", "reviewTime"]]
#rev = rev.rename(columns={"reviewerID": "user_id", "asin": "item_ids", "overall": "ratings", "reviewTime": "timestamps"})
rev['timestamps'] = pd.to_datetime(rev['reviewTime'], format='%m %d, %Y')

user_ids = rev['reviewerID'][:]
item_ids = rev['asin'][:]

retain_user_ids = _filter_by_count(user_ids, min_user_interactions)
retain_item_ids = _filter_by_count(item_ids, min_item_interactions)

rev = rev[rev["reviewerID"].isin(retain_user_ids)]
rev = rev[rev["asin"].isin(retain_item_ids)]


In the following, the models will be tested to obtain the model with the lowest RMSE.

In [42]:
benchmark = []
# Iterate over all algorithms
data = Dataset.load_from_df(rev[['reviewerID', 'asin', 'overall']], reader)
algorithms = [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]

print ("Attempting: ", str(algorithms), '\n\n\n')

for algorithm in algorithms:
    print("Starting: " ,str(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    # results = cross_validate(algorithm, data, measures=['RMSE','MAE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    print("Done: " ,str(algorithm), "\n\n")

print ('\n\tDONE\n')
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
surprise_results.sort_values(by=['test_rmse'])

Attempting:  [<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x000001E286C7BAC0>, <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x000001E286C7BD90>, <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x000001E289442FA0>, <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x000001E286CEB6A0>, <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x000001E286CEB7F0>, <surprise.prediction_algorithms.knns.KNNBaseline object at 0x000001E286CEB130>, <surprise.prediction_algorithms.knns.KNNBasic object at 0x000001E286CEB910>, <surprise.prediction_algorithms.knns.KNNWithMeans object at 0x000001E286CEB220>, <surprise.prediction_algorithms.knns.KNNWithZScore object at 0x000001E286CEB4F0>, <surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x000001E286CEB520>, <surprise.prediction_algorithms.co_clustering.CoClustering object at 0x000001E286CEB430>] 



Starting:  <surprise.prediction_algorithms.

,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,1.051699,5.592365,0.363007
KNNBaseline,1.052610,3.897214,2.807301
SVD,1.074405,2.242446,0.153748
KNNBasic,1.092703,3.839198,2.437512
KNNWithMeans,1.135877,4.490177,2.547674
KNNWithZScore,1.138528,5.942605,2.631212
SlopeOne,1.141045,1.800823,0.143559
CoClustering,1.169893,3.710833,0.295859
BaselineOnly,1.201018,0.095710,0.095328


As can be seen in the results above, KNNBaseline and SVDpp are the models with the lowest error and are, therefore, the two models which will be used in the following procedures.

Once the two best models are chosen, a tuning of the parameters will be performed and the models will be compared again.

In [43]:
def tune_kknbaseline(reviews):
    
    reviewsR2 = reviews
    data = Dataset.load_from_df(reviewsR2[['reviewerID', 'asin', 'overall']], reader)
    sim_options = {
        "name": ["msd", "cosine"],
        "min_support": [1, 2, 3],
        "user_based": [False, True],
    }
    param_grid = {"sim_options": sim_options}
    gs_knn = GridSearchCV(KNNBaseline, param_grid, measures=["rmse", "mae"], cv=3)
    gs_knn.fit(data)

    #print(gs.best_score["rmse"])
    #print(gs.best_params["rmse"])
    return(gs_knn)

def tune_SVDpp(reviews):
    
    reviewsR2 = reviews
    data = Dataset.load_from_df(reviewsR2[['reviewerID', 'asin', 'overall']], reader)
    param_grid = {
        "n_epochs": [30, 35],
        "lr_all": [0.009, 0.012],
        "reg_all": [0.01, 0.02]
    }
    gs_svd = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=3)
    gs_svd.fit(data)

    return(gs_svd)

gs_svd = tune_SVDpp(rev)
Svd_params = gs_svd.best_params
Svd_rmse = gs_svd.best_score["rmse"]

gs_knn = tune_kknbaseline(rev)
Knn_params = gs_knn.best_params
Knn_rmse = gs_knn.best_score["rmse"]


Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

In [63]:
print(Svd_rmse)
print(Svd_params)
print(Knn_rmse)
print(Knn_params)

1.0184111378664402
{'rmse': {'n_epochs': 35, 'lr_all': 0.012, 'reg_all': 0.02}, 'mae': {'n_epochs': 35, 'lr_all': 0.012, 'reg_all': 0.01}}
1.0249652463011878
{'rmse': {'sim_options': {'name': 'msd', 'min_support': 2, 'user_based': False}}, 'mae': {'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}}


We choose the model with the lowest RMSE (SVDpp) and train it with the selected parameters.

In [46]:
model = SVDpp(n_epochs = 35, lr_all = 0.009, reg_all = 0.01)
trainingSet = data.build_full_trainset()
model.fit(trainingSet)

Once the final model is trained, we proceed to create the function that will give the 50 final recommendations for each selected client.

The function consists on two principal parts:

- Firstly, the recommendations for each product reviewed by the client are obtained with the function "get_recommendations", created in the previous section. For each product, a list of 100 recommendations will be created. From all the recommendations a group of the 200 ones with the highest score will be selected. This score is based on the review given by the client to the product to which the recommendation is related to and the similarity scores calculated before. 

- For these selection of products, it will be predicted which review (from 0 to 5) would the client give to those products. This will be estimated with the SVDpp model. Once the estimation is calculated for every product, the 50 products with a higher estimation are selected as the final recommendation for the client.  

In [93]:
zzz = get_recommendations('B0148B7EJ6',50, cosine_reviews,'reviews') #example
cols = list(zzz.columns)+['p_score']
cols_F = ["reviewerID" ,"Asin", "est"]
types = ['reviews', 'soup']
reviewers_list = ['AXRC1EH0U2WZ9','A3J5XBZ9ZNEZOC', 'AKW4JIYF5I08K', 'A4455JHTQFWAO', 'A2VHVODX6A92OA',
                  'A3TMN244T2LJID', 'A3KA46MLPT89OH', 'A1RVDD4O9OPHAM', 'A3OP70LPLALBKN', 'A2NIO806C6LXSJ']
reviews_final = rev[rev['reviewerID'].isin(reviewers_list)]

def recommendation(reviews):
    client_p = pd.DataFrame(columns=cols)
    Final_recommendations = pd.DataFrame(columns=cols_F)
    for i in reviews['reviewerID'].unique():
        reviewsS = reviews[reviews['reviewerID'] == i]
        p = reviewsS['asin'].unique()
        client_p = pd.DataFrame(columns=cols)
        for tipo in types:
            if tipo == 'reviews':
                cc = cosine_reviews
            else:
                cc = cosine_soup
            for r in p:
                if r in rev['asin'].values:
                    re = get_recommendations(r,50,cc,tipo)
                    re['p_score'] = reviewsS['overall'][reviewsS['asin'] == r].iloc[0] #score que le ha dado al producto por el cual lo recomienda
                    client_p = client_p.append(re)
        client_p['p_s'] = client_p['score']*client_p['p_score']*client_p['nlp_similarity']
        client_p = client_p.sort_values('p_s', ascending = False)
        client_p = client_p[~client_p['asin'].isin(p)]
        client_p = client_p[['asin']]
        client_p = client_p.drop_duplicates()
        prueba = client_p.head(200)
        print(len(prueba))
        prueba['est'] = 'NP'
        for z in prueba['asin']:
            x = model.predict(i, z, 5).est
            prueba['est'][prueba['asin'] == z] = x
        prueba = prueba.sort_values('est', ascending = False).head(50)
        prueba.insert(0, 'reviewerID', i)
        Final_recommendations = Final_recommendations.append(prueba)
    return(Final_recommendations)

    

In the table "Final_recommendations", the 50 recommendations for the 10 selected clients are stored.

In [94]:
Final_recommendations = recommendation(reviews_final)

200
200
200
200
200
200
200
200
200
200


In [99]:
Final_recommendations.head(2)


,reviewerID,Asin,est,asin
470,A3OP70LPLALBKN,NaN,5,B001XOQTSE
11367,A3OP70LPLALBKN,NaN,5,B00ZW3SCF0
